In [1]:
from urllib.request import urlopen
from urllib.parse import urlparse
from bs4 import BeautifulSoup
import re
import datetime
import random
import pandas as pd

In [2]:
# Web to scrape, checking the structure of the pagers and detect total of pagers

startingPage = "https://teestamosbuscando.pe/"
html = urlopen(startingPage)
bsObj = BeautifulSoup(html, 'lxml')

for pager in bsObj.findAll("li"):
    print(pager)

<li class="first leaf"><a class="active" href="/" title="">Inicio</a></li>
<li class="leaf"><a href="/desaparecidos" title="">Desaparecidos</a></li>
<li class="last leaf"><a href="/encontrados" title="">Encontrados</a></li>
<li>
<div class="views-field views-field-field-imagen"> <div class="field-content"><img alt="" height="573" src="https://teestamosbuscando.pe/sites/default/files/slider/img/banner3.png" typeof="foaf:Image" width="2480"/></div> </div></li>
<li class="pager-current first">1</li>
<li class="pager-item"><a href="/node?page=1" title="Ir a la página 2">2</a></li>
<li class="pager-item"><a href="/node?page=2" title="Ir a la página 3">3</a></li>
<li class="pager-item"><a href="/node?page=3" title="Ir a la página 4">4</a></li>
<li class="pager-item"><a href="/node?page=4" title="Ir a la página 5">5</a></li>
<li class="pager-item"><a href="/node?page=5" title="Ir a la página 6">6</a></li>
<li class="pager-item"><a href="/node?page=6" title="Ir a la página 7">7</a></li>
<li cl

In [3]:
# Generating a list of missed people until the last pager
#*****************************
# Change the number of pagers
#*****************************

first_page = "node"
pager = "node?page="
pagers = 3

missedPagers = [startingPage + pager + str(i) for i in list(range(1,pagers))]
missedPagers.insert(0,startingPage + first_page)

missedLinks = []

for pager in missedPagers:
    html = urlopen(pager)
    bsObj = BeautifulSoup(html, 'lxml')
    for link in bsObj.findAll("a", href = re.compile("^(/desaparecido/)")):
        if link.attrs['href'] is not None:
            if link.attrs['href'] not in missedLinks:
                missedLinks.append(link.attrs['href'])

missedLinks = [startingPage[:-1] + link for link in missedLinks]

In [4]:
# Collect the details per person in a dataframe

from timeit import default_timer as timer

missedPages = missedLinks
missedList = []

start = timer() 

for missedPage in missedPages:
    featureText = []
    featureLabel = []
    ubigeo = ''
    html = urlopen(missedPage)
    bsObj = BeautifulSoup(html, 'lxml')
    featureObj = bsObj.find(id = "zone-content")
    
    for item in featureObj('div', attrs={'class':'field-item'}):
        featureText.append(item.get_text())
    featureText.pop(0)
    
    for item in featureObj('div', attrs={'class':'field-label'}):
        featureLabel.append(re.sub(':\xa0','',item.get_text()).replace(' ','_').lower())
    
    missed = dict(zip(featureLabel,featureText))
    
    for item in featureObj.findAll('li'):
        ubigeo = ubigeo +  ',' + item.text
    missed['ubigeo'] = ubigeo[1:]

    missedList.append(missed)

end = timer() 

print(end - start)

3756.7594384319964


In [26]:
# Downloading the dataframe generated to a csv file 

import os
# Exportando los datos a csv
path = os.getcwd()
os.chdir(path +'/missed')

dropColumns = ['denunciante','edad_denunciante','sexo_denunciante','cara','discapacidad',
               'telefono_denunciante','circunstancias','shorten_url','cejas','tez','vestimenta',
               'señas_particulares','dirección','dni','fecha_de_nacimiento','nro_de_denuncia',
               'observaciones','parentesco_denunciante','boca','cabellos','nariz','ojos',
               'contextura']

pd.DataFrame(missedList).drop(dropColumns, axis = 1).to_csv('missed.csv')